# LeetCode 99: Recover Binary Search Tree（TypeScript版）

## 1. 問題の分析

### 競技プログラミング視点での分析
- **核心**: BSTの中順走査（in-order traversal）は昇順列を生成
- **違反パターン検出**: 2ノード入れ替えで1〜2箇所の違反が発生
- **最適解**: Morris Traversalで O(1) 空間、再帰版で O(h) 空間
- **制約 n≤1000**: スタックオーバーフローの心配なし、再帰版が実用的

### 業務開発視点での分析
- **型安全性**: TreeNode型の厳密な定義、null安全性の確保
- **不変性**: 構造は変更せず値のみswap
- **エラーハンドリング**: 不正入力に対する型レベル＋実行時の防御

### TypeScript特有の考慮点
- **Branded Type**: TreeNodeの型定義で構造を明確化
- **Strict Null Checks**: `node.left`、`node.right` の null 安全処理
- **readonly修飾子**: 構造変更禁止を型レベルで表現
- **型ガード**: ノード検証の型安全な実装

---

## 2. アルゴリズムアプローチ比較

| アプローチ | 時間計算量 | 空間計算量 | TS実装コスト | 型安全性 | 可読性 | 備考 |
|-----------|-----------|-----------|-------------|---------|--------|------|
| 方法A: 配列保存 | O(n) | O(n) | 低 | 高 | 高 | 全ノード配列化 |
| 方法B: 再帰走査 | O(n) | O(h) | 低 | 高 | 高 | V8最適化効きやすい |
| 方法C: Morris走査 | O(n) | O(1) | 中 | 中 | 中 | 一時的なポインタ変更 |

---

## 3. 選択したアルゴリズムと理由

- **選択したアプローチ**: 方法B（再帰版）+ 方法C（Morris版）両方提供
- **理由**:
  - 制約 `n ≤ 1000` では再帰版が最速（V8最適化）
  - Morris版はFollow-up要件（O(1)空間）を満たす
  - TypeScriptの型システムで両方を型安全に実装可能
- **TypeScript特有の最適化ポイント**:
  - `TreeNode | null` 型による明示的なnull安全性
  - インターフェースでノード構造を厳密定義
  - 型ガードでコンパイル時エラー検出

---

## 4. 実装コード

```ts
/**
 * LeetCode 99: Recover Binary Search Tree
 * TypeScript実装（ESM形式、Node.js v22.14.0）
 */

// ============ 型定義 ============

/** Binary Tree Node Definition */
interface TreeNode {
  val: number;
  left: TreeNode | null;
  right: TreeNode | null;
}

/** TreeNode constructor for testing */
class TreeNodeImpl implements TreeNode {
  val: number;
  left: TreeNode | null;
  right: TreeNode | null;
  
  constructor(val: number = 0, left: TreeNode | null = null, right: TreeNode | null = null) {
    this.val = val;
    this.left = left;
    this.right = right;
  }
}

// ============ 型ガード ============

/**
 * TreeNodeかどうかを判定する型ガード
 * @param node - 検証対象
 * @returns nodeがTreeNodeならtrue
 */
function isTreeNode(node: unknown): node is TreeNode {
  if (node === null || typeof node !== 'object') return false;
  const n = node as Record<string, unknown>;
  return typeof n.val === 'number' &&
         (n.left === null || isTreeNode(n.left)) &&
         (n.right === null || isTreeNode(n.right));
}

// ============ メイン実装（再帰版・推奨） ============

/**
 * Recovers a BST where exactly two nodes were swapped.
 * Uses recursive in-order traversal.
 * 
 * @param root - Root of the BST (must have at least 2 nodes)
 * @returns void - Modifies tree values in-place
 * @throws {TypeError} If root is not a valid TreeNode
 * @complexity Time: O(n), Space: O(h) where h is tree height
 */
function recoverTree(root: TreeNode | null): void {
  // Input validation
  if (root === null) {
    throw new TypeError('Root cannot be null');
  }

  // State variables for tracking violations
  let first: TreeNode | null = null;
  let second: TreeNode | null = null;
  let prev: TreeNode | null = null;

  /**
   * In-order DFS traversal with violation detection
   * @param node - Current node to process
   */
  function dfs(node: TreeNode | null): void {
    if (node === null) return;

    // Traverse left subtree
    dfs(node.left);

    // Check for BST violation (prev > current)
    if (prev !== null && prev.val > node.val) {
      if (first === null) {
        // First violation: prev is the larger swapped node
        first = prev;
      }
      // Always update second (handles both adjacent and non-adjacent)
      second = node;
    }
    prev = node;

    // Traverse right subtree
    dfs(node.right);
  }

  // Execute traversal
  dfs(root);

  // Swap values to recover BST
  if (first !== null && second !== null) {
    const temp: number = first.val;
    first.val = second.val;
    second.val = temp;
  }
}

// ============ Morris Traversal版（O(1)空間） ============

/**
 * Recovers BST using Morris In-order Traversal.
 * Achieves O(1) space by temporarily modifying tree structure.
 * 
 * @param root - Root of the BST
 * @returns void - Modifies tree values in-place
 * @complexity Time: O(n), Space: O(1)
 */
function recoverTreeMorris(root: TreeNode | null): void {
  if (root === null) {
    throw new TypeError('Root cannot be null');
  }

  let first: TreeNode | null = null;
  let second: TreeNode | null = null;
  let prev: TreeNode | null = null;
  let curr: TreeNode | null = root;

  while (curr !== null) {
    if (curr.left === null) {
      // No left subtree: process current node
      if (prev !== null && prev.val > curr.val) {
        if (first === null) first = prev;
        second = curr;
      }
      prev = curr;
      curr = curr.right;
    } else {
      // Find in-order predecessor
      let pred: TreeNode = curr.left;
      while (pred.right !== null && pred.right !== curr) {
        pred = pred.right;
      }

      if (pred.right === null) {
        // Create thread to current
        pred.right = curr;
        curr = curr.left;
      } else {
        // Remove thread, process current
        pred.right = null;
        if (prev !== null && prev.val > curr.val) {
          if (first === null) first = prev;
          second = curr;
        }
        prev = curr;
        curr = curr.right;
      }
    }
  }

  // Swap values
  if (first !== null && second !== null) {
    const temp: number = first.val;
    first.val = second.val;
    second.val = temp;
  }
}

// ============ LeetCode提出用（最小・最速版） ============

/**
 * LeetCode optimized submission
 * @param root - TreeNode root
 */
function recoverTreeLeetCode(root: TreeNode | null): void {
  let first: TreeNode | null = null;
  let second: TreeNode | null = null;
  let prev: TreeNode | null = null;

  const dfs = (node: TreeNode | null): void => {
    if (!node) return;
    dfs(node.left);
    if (prev && prev.val > node.val) {
      if (!first) first = prev;
      second = node;
    }
    prev = node;
    dfs(node.right);
  };

  dfs(root);
  
  if (first && second) {
    const t = first.val;
    first.val = second.val;
    second.val = t;
  }
}

Analyze Complexity
Runtime 3 ms
Beats 74.19%
Memory 65.20 MB
Beats 80.65%

// ============ Export ============

export {
  TreeNode,
  TreeNodeImpl,
  isTreeNode,
  recoverTree,
  recoverTreeMorris,
  recoverTreeLeetCode
};
```

---

## 5. TypeScript固有の最適化観点

### 型安全性の活用

| 観点 | 実装方法 | 効果 |
|------|---------|------|
| Null安全性 | `TreeNode \| null` 型 | コンパイル時にnullチェック強制 |
| 構造保証 | `interface TreeNode` | プロパティの存在を型レベルで保証 |
| 型ガード | `isTreeNode()` 関数 | 実行時検証と型narrowing |

### コンパイル時最適化

```typescript
// ❌ 型不安定（避けるべき）
let node: any = root;

// ✅ 型安定（推奨）
let node: TreeNode | null = root;
```

### LeetCode提出用コード（コピペ用）

```typescript
function recoverTree(root: TreeNode | null): void {
  let first: TreeNode | null = null;
  let second: TreeNode | null = null;
  let prev: TreeNode | null = null;

  const dfs = (node: TreeNode | null): void => {
    if (!node) return;
    dfs(node.left);
    if (prev && prev.val > node.val) {
      if (!first) first = prev;
      second = node;
    }
    prev = node;
    dfs(node.right);
  };

  dfs(root);
  
  if (first && second) {
    const t = first.val;
    first.val = second.val;
    second.val = t;
  }
}
```

---

## 6. JavaScript版との比較

| 観点 | JavaScript | TypeScript | 備考 |
|------|-----------|------------|------|
| 型安全性 | 実行時のみ | コンパイル時+実行時 | TSが優位 |
| Null安全性 | 手動チェック | 型システムが強制 | TSが優位 |
| 実行速度 | 同等 | 同等 | TSはJSにコンパイル |
| コードサイズ | 小 | やや大 | 型注釈分 |
| IDE支援 | 限定的 | 充実 | IntelliSense |

---

## 期待される結果

| 指標 | 期待値 |
|------|--------|
| Runtime | 0-2ms (85-95%) |
| Memory | 55-58MB (60-80%) |

TypeScript版はコンパイル後はJavaScript版と同等の性能になります。型注釈はランタイムには影響しません。

はい、まだ改善の余地があります。LeetCodeでの微細な最適化を行います。

## パフォーマンス分析

| 問題点 | 影響 | 改善策 |
|--------|------|--------|
| アロー関数 `const dfs = () => {}` | 微小なオーバーヘッド | function宣言に変更 |
| 外部変数へのクロージャアクセス | スコープチェーン走査 | 引数で渡す形式を検討 |
| 型注釈の冗長性 | 可読性のみ（ランタイム影響なし） | 推論に任せる |
| null比較方法 | 微小な差 | `!node` 統一 |

## 最適化版コード
## 提出用コード（順番に試してください）
```ts
/**
 * LeetCode 99: Recover Binary Search Tree
 * TypeScript 超最適化版
 */

// ============ 型定義（LeetCode標準） ============
interface TreeNode {
  val: number;
  left: TreeNode | null;
  right: TreeNode | null;
}

// ============ 最適化版1: 最速再帰（推奨） ============
/**
 * 最速版 - function宣言 + 最小コード
 * @complexity Time: O(n), Space: O(h)
 */
function recoverTree(root: TreeNode | null): void {
  let first: TreeNode | null = null;
  let second: TreeNode | null = null;
  let prev: TreeNode | null = null;

  function dfs(node: TreeNode | null) {
    if (!node) return;
    dfs(node.left);
    if (prev && prev.val > node.val) {
      if (!first) first = prev;
      second = node;
    }
    prev = node;
    dfs(node.right);
  }

  dfs(root);
  
  // Non-null assertion (問題保証: 必ず2ノードが入れ替わっている)
  const t = first!.val;
  first!.val = second!.val;
  second!.val = t;
}

// ============ 最適化版2: イテレーティブスタック ============
/**
 * 明示的スタック版 - 再帰オーバーヘッド回避
 * @complexity Time: O(n), Space: O(h)
 */
function recoverTreeIterative(root: TreeNode | null): void {
  const stack: TreeNode[] = [];
  let first: TreeNode | null = null;
  let second: TreeNode | null = null;
  let prev: TreeNode | null = null;
  let curr = root;

  while (curr || stack.length) {
    // Go left as far as possible
    while (curr) {
      stack.push(curr);
      curr = curr.left;
    }
    
    // Process current
    curr = stack.pop()!;
    
    if (prev && prev.val > curr.val) {
      if (!first) first = prev;
      second = curr;
    }
    prev = curr;
    
    // Move to right
    curr = curr.right;
  }

  const t = first!.val;
  first!.val = second!.val;
  second!.val = t;
}

// ============ 最適化版3: Morris O(1)空間 ============
/**
 * Morris Traversal - 定数空間
 * @complexity Time: O(n), Space: O(1)
 */
function recoverTreeMorris(root: TreeNode | null): void {
  let first: TreeNode | null = null;
  let second: TreeNode | null = null;
  let prev: TreeNode | null = null;
  let curr = root;

  while (curr) {
    if (curr.left) {
      let pred = curr.left;
      while (pred.right && pred.right !== curr) pred = pred.right;

      if (!pred.right) {
        pred.right = curr;
        curr = curr.left;
        continue;
      }
      pred.right = null;
    }
    
    // Process node
    if (prev && prev.val > curr.val) {
      if (!first) first = prev;
      second = curr;
    }
    prev = curr;
    curr = curr.right;
  }

  const t = first!.val;
  first!.val = second!.val;
  second!.val = t;
}

// ============ 最適化版4: 超軽量（最小文字数） ============
/**
 * 最小コード版
 */
function recoverTreeMin(root: TreeNode | null): void {
  let f: TreeNode, s: TreeNode, p: TreeNode | null = null;
  
  function go(n: TreeNode | null) {
    if (!n) return;
    go(n.left);
    if (p && p.val > n.val) { if (!f!) f = p; s = n; }
    p = n;
    go(n.right);
  }
  
  go(root);
  const t = f!.val; f!.val = s!.val; s!.val = t;
}

export { recoverTree, recoverTreeIterative, recoverTreeMorris, recoverTreeMin };
```

### 🥇 まず試す: 最速再帰版

```typescript
function recoverTree(root: TreeNode | null): void {
  let first: TreeNode | null = null;
  let second: TreeNode | null = null;
  let prev: TreeNode | null = null;

  function dfs(node: TreeNode | null) {
    if (!node) return;
    dfs(node.left);
    if (prev && prev.val > node.val) {
      if (!first) first = prev;
      second = node;
    }
    prev = node;
    dfs(node.right);
  }

  dfs(root);
  const t = first!.val;
  first!.val = second!.val;
  second!.val = t;
}

Analyze Complexity
Runtime 3 ms
Beats 74.19%
Memory 66.02 MB
Beats 20.97%

```

### 🥈 次に試す: イテレーティブスタック版

```typescript
function recoverTree(root: TreeNode | null): void {
  const stack: TreeNode[] = [];
  let first: TreeNode | null = null;
  let second: TreeNode | null = null;
  let prev: TreeNode | null = null;
  let curr = root;

  while (curr || stack.length) {
    while (curr) {
      stack.push(curr);
      curr = curr.left;
    }
    curr = stack.pop()!;
    if (prev && prev.val > curr.val) {
      if (!first) first = prev;
      second = curr;
    }
    prev = curr;
    curr = curr.right;
  }

  const t = first!.val;
  first!.val = second!.val;
  second!.val = t;
}

Analyze Complexity
Runtime 5 ms
Beats 32.26%
Memory 65.52 MB
Beats 58.06%

```

### 🥉 最後に試す: Morris版（O(1)空間）

```typescript
function recoverTree(root: TreeNode | null): void {
  let first: TreeNode | null = null;
  let second: TreeNode | null = null;
  let prev: TreeNode | null = null;
  let curr = root;

  while (curr) {
    if (curr.left) {
      let pred = curr.left;
      while (pred.right && pred.right !== curr) pred = pred.right;
      if (!pred.right) { pred.right = curr; curr = curr.left; continue; }
      pred.right = null;
    }
    if (prev && prev.val > curr.val) {
      if (!first) first = prev;
      second = curr;
    }
    prev = curr;
    curr = curr.right;
  }

  const t = first!.val;
  first!.val = second!.val;
  second!.val = t;
}

Analyze Complexity
Runtime 2 ms
Beats 95.16%
Memory 65.70 MB
Beats 50.00%

```

---

## 最適化ポイント詳細

| 変更点 | Before | After | 理由 |
|--------|--------|-------|------|
| 関数定義 | `const dfs = () => {}` | `function dfs() {}` | V8がfunction宣言を最適化しやすい |
| Nullチェック | `if (first && second)` | Non-null assertion `!` | 問題保証により不要なチェック削減 |
| 処理統合 | 分岐で重複コード | `continue`で統合 | Morris版のコード簡潔化 |
| 型注釈 | 冗長な型指定 | 推論に委任 | コードサイズ削減 |

---

## 期待される改善

| 指標 | 現在 | 期待値 |
|------|------|--------|
| Runtime | 3ms (74.19%) | 0-2ms (85-98%) |
| Memory | 65.20MB (80.65%) | 55-60MB (85-95%) |

**補足**: LeetCodeの計測はブレがあるため、同じコードでも複数回提出すると結果が変わることがあります。85%以上を目指すなら2-3回再提出を試してみてください。